# Modelo de Regressão - Predição do Peso de Peixes

Este _notebook_ implementa o treinamento de um modelo de regressão para prever o peso de um peixe com base em um conjunto de atributos. O conjunto de dados utilizado é o `regression_fish.csv`, que pode ser acessado na pasta `data`. Ele foi obtido através da plataforma Kaggle, [neste _link_](https://www.kaggle.com/datasets/vipullrathod/fish-market).

As redes foram treinadas com diferentes configurações para avaliar o desempenho dos modelos com diferentes funções de ativação, épocas e _hidden layers_. Para o treinamento dos modelos, a biblioteca de Rede Neural Artificial desenvolvida pelos membros do grupo foi utilizada. Mais precisamente, a classe `NeuralNetwork`, a qual representa a rede, foi instanciada e os modelos foram treinados através dos métodos implementados pela classe.

## Dependências

Esta célula é responsável pela importação das dependências utilizadas para os treinamentos dos modelos. Foram utilizadas a classe `NeuralNetwork`, `polars` e `numpy` para manipulação de dados, e `scikit-learn` para o pré-processamento e divisão dos dados.

In [29]:
from neural_lib.neural_network import NeuralNetwork
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

## Carregamento e pré-processamento dos dados

Aqui, os dados do _dataset_ `regression_fish.csv` são carregados. A coluna 'Weight' é definida como a variável alvo (y). As outras colunas são as características (X).

As características numéricas contínuas (`numerical_features`) são escalonadas usando o `StandardScaler` para normalizar os dados.

In [30]:
def load_and_preprocess(csv_path):
    df = pl.read_csv(csv_path)

    y = df.select('Weight').to_numpy()
    X_raw = df.drop('Weight')

    scaler = StandardScaler()
    X = scaler.fit_transform(X_raw.to_numpy())

    return X, y

## Definição do modelo e funções auxiliares

A função `train_model` cria e treina uma instância da rede neural para regressão.

A função `evaluate_model` avalia o desempenho do modelo usando o Erro Quadrático Médio (MSE) e o R-quadrado (R²) nos dados de teste.

In [31]:
def train_model(X_train, y_train, hidden_layers, epochs, activation_function, learning_rate):
    input_size = X_train.shape[1]
    neural_network = NeuralNetwork(
        input_size=input_size,
        hidden_sizes=hidden_layers,
        activation=activation_function,
        output_size=1,
        output_activation='identity',
        loss='mse'
    )

    neural_network.train(X_train, y_train, epochs, learning_rate)
    return neural_network

def evaluate_model(y_true, y_pred):
    """
    Avalia o modelo de regressão calculando R² e RMSE
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    return r2, rmse

## Treinamento e avaliação dos modelos

A função `get_params()` define diferentes conjuntos de hiperparâmetros. Os dados são divididos e a variável alvo `y` é escalada para estabilizar o treinamento. Após o treinamento, as predições são revertidas para a escala original antes de calcular as métricas.

In [32]:
def get_params():
    return [
        {'hidden_layers': [64, 32], 'activation_function': 'tanh', 'epochs': 1000, 'learning_rate': 0.001},
        {'hidden_layers': [8, 2], 'activation_function': 'relu', 'epochs': 1000, 'learning_rate': 0.001},
        {'hidden_layers': [128, 64, 32], 'activation_function': 'tanh', 'epochs': 1500, 'learning_rate': 0.001},
        {'hidden_layers': [128, 64, 32], 'activation_function': 'relu', 'epochs': 1500, 'learning_rate': 0.001},
    ]

X, y = load_and_preprocess('./../data/regression_fish.csv')

# Redimensiona y para o formato (n_samples, 1) para o scaler
y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Cria e treina um scaler para a variável alvo 'y' APENAS com os dados de treino
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

for params in get_params():
    hidden_layers = params['hidden_layers']
    epochs = params['epochs']
    activation_function = params['activation_function']
    learning_rate = params['learning_rate']

    neural_network = train_model(
        X_train, y_train_scaled,
        hidden_layers,
        epochs,
        activation_function,
        learning_rate
    )
    
    predictions_scaled = neural_network.predict(X_test)
    predictions_original_scale = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1))

    r2, rmse = evaluate_model(y_test, predictions_original_scale)
    
    print(f'Hidden Layers: {hidden_layers}, Epochs: {epochs}, Activation: {activation_function}, LR: {learning_rate} ->  RMSE: {rmse:.2f}, R²: {r2:.2f}')

Hidden Layers: [64, 32], Epochs: 1000, Activation: tanh, LR: 0.001 ->  RMSE: 92.45, R²: 0.94
Hidden Layers: [8, 2], Epochs: 1000, Activation: relu, LR: 0.001 ->  RMSE: 208.17, R²: 0.70
Hidden Layers: [128, 64, 32], Epochs: 1500, Activation: tanh, LR: 0.001 ->  RMSE: 87.89, R²: 0.95
Hidden Layers: [128, 64, 32], Epochs: 1500, Activation: relu, LR: 0.001 ->  RMSE: 67.20, R²: 0.97
